# Delivery Duration Prediction

In [317]:
import warnings
warnings.filterwarnings('ignore')

In [318]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Loading Dataset

In [319]:
df = pd.read_csv(r'https://raw.githubusercontent.com/akash-ostwal/Machine-Learning/refs/heads/main/Project1-Regression_Delivery_Duration_Prediction/Historical_Dataset.csv')

In [320]:
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,22:24:17,23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,21:49:25,22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,20:39:28,21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,21:21:45,22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,2:40:36,3:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


In [321]:
df.columns = df.columns.str.upper()
df.columns

Index(['MARKET_ID', 'CREATED_AT', 'ACTUAL_DELIVERY_TIME', 'STORE_ID',
       'STORE_PRIMARY_CATEGORY', 'ORDER_PROTOCOL', 'TOTAL_ITEMS', 'SUBTOTAL',
       'NUM_DISTINCT_ITEMS', 'MIN_ITEM_PRICE', 'MAX_ITEM_PRICE',
       'TOTAL_ONSHIFT_DASHERS', 'TOTAL_BUSY_DASHERS',
       'TOTAL_OUTSTANDING_ORDERS', 'ESTIMATED_ORDER_PLACE_DURATION',
       'ESTIMATED_STORE_TO_CONSUMER_DRIVING_DURATION'],
      dtype='object')

### Data quality checks

In [322]:
df.shape

(197428, 16)

In [323]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   MARKET_ID                                     196441 non-null  float64
 1   CREATED_AT                                    197428 non-null  object 
 2   ACTUAL_DELIVERY_TIME                          197421 non-null  object 
 3   STORE_ID                                      197428 non-null  int64  
 4   STORE_PRIMARY_CATEGORY                        192668 non-null  object 
 5   ORDER_PROTOCOL                                196433 non-null  float64
 6   TOTAL_ITEMS                                   197428 non-null  int64  
 7   SUBTOTAL                                      197428 non-null  int64  
 8   NUM_DISTINCT_ITEMS                            197428 non-null  int64  
 9   MIN_ITEM_PRICE                                19

In [324]:
# Checking duplicated data
df.duplicated().sum()

np.int64(0)

In [325]:
df = df.drop_duplicates()

In [326]:
# Checking missing data
missing_counts = df.isna().sum()
missing_percentage = (missing_counts/df.shape[0]*100).round(2)

In [327]:
missing_summary = pd.DataFrame({
    "missing_counts": missing_counts,
    "missing_percentage": missing_percentage
})
missing_summary

,missing_counts,missing_percentage
MARKET_ID,987,0.50
CREATED_AT,0,0.00
ACTUAL_DELIVERY_TIME,7,0.00
STORE_ID,0,0.00
STORE_PRIMARY_CATEGORY,4760,2.41
ORDER_PROTOCOL,995,0.50
TOTAL_ITEMS,0,0.00
SUBTOTAL,0,0.00
NUM_DISTINCT_ITEMS,0,0.00
MIN_ITEM_PRICE,0,0.00


In [328]:
# For now we are droping the missing data which is below 0.5
df = df.dropna(subset=['MARKET_ID','ACTUAL_DELIVERY_TIME','ORDER_PROTOCOL','ESTIMATED_STORE_TO_CONSUMER_DRIVING_DURATION'])

In [329]:
df.isna().sum()

MARKET_ID                                           0
CREATED_AT                                          0
ACTUAL_DELIVERY_TIME                                0
STORE_ID                                            0
STORE_PRIMARY_CATEGORY                           3995
ORDER_PROTOCOL                                      0
TOTAL_ITEMS                                         0
SUBTOTAL                                            0
NUM_DISTINCT_ITEMS                                  0
MIN_ITEM_PRICE                                      0
MAX_ITEM_PRICE                                      0
TOTAL_ONSHIFT_DASHERS                           16107
TOTAL_BUSY_DASHERS                              16107
TOTAL_OUTSTANDING_ORDERS                        16107
ESTIMATED_ORDER_PLACE_DURATION                      0
ESTIMATED_STORE_TO_CONSUMER_DRIVING_DURATION        0
dtype: int64

In [330]:
# Converring data type of CREATED_AT & ACTUAL_DELIVERY_TIME columns.
df['CREATED_AT'] = pd.to_timedelta(df['CREATED_AT'])
df['ACTUAL_DELIVERY_TIME'] = pd.to_timedelta(df['ACTUAL_DELIVERY_TIME'])

df['DELIVERY_DURATION_SEC'] = (df['ACTUAL_DELIVERY_TIME'] - df['CREATED_AT']).dt.total_seconds()

In [331]:
df.dtypes

MARKET_ID                                               float64
CREATED_AT                                      timedelta64[ns]
ACTUAL_DELIVERY_TIME                            timedelta64[ns]
STORE_ID                                                  int64
STORE_PRIMARY_CATEGORY                                   object
ORDER_PROTOCOL                                          float64
TOTAL_ITEMS                                               int64
SUBTOTAL                                                  int64
NUM_DISTINCT_ITEMS                                        int64
MIN_ITEM_PRICE                                            int64
MAX_ITEM_PRICE                                            int64
TOTAL_ONSHIFT_DASHERS                                   float64
TOTAL_BUSY_DASHERS                                      float64
TOTAL_OUTSTANDING_ORDERS                                float64
ESTIMATED_ORDER_PLACE_DURATION                            int64
ESTIMATED_STORE_TO_CONSUMER_DRIVING_DURA

In [332]:
df.head()

,MARKET_ID,CREATED_AT,ACTUAL_DELIVERY_TIME,STORE_ID,STORE_PRIMARY_CATEGORY,ORDER_PROTOCOL,TOTAL_ITEMS,SUBTOTAL,NUM_DISTINCT_ITEMS,MIN_ITEM_PRICE,MAX_ITEM_PRICE,TOTAL_ONSHIFT_DASHERS,TOTAL_BUSY_DASHERS,TOTAL_OUTSTANDING_ORDERS,ESTIMATED_ORDER_PLACE_DURATION,ESTIMATED_STORE_TO_CONSUMER_DRIVING_DURATION,DELIVERY_DURATION_SEC
0,1.0,0 days 22:24:17,0 days 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,3779.0
1,2.0,0 days 21:49:25,0 days 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,4024.0
2,3.0,0 days 20:39:28,0 days 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0,1781.0
3,3.0,0 days 21:21:45,0 days 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0,3075.0
4,3.0,0 days 02:40:36,0 days 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0,2390.0


In [333]:
df['CREATED_AT'].mode()

0   0 days 02:27:40
Name: CREATED_AT, dtype: timedelta64[ns]

In [334]:
df['ACTUAL_DELIVERY_TIME'].mode()

0   0 days 03:27:25
Name: ACTUAL_DELIVERY_TIME, dtype: timedelta64[ns]

### Data Cleaning & Data preprocessing

In [335]:
# Organizing the dataset into numerical, categorical, and datetime columns.
cat  = df.select_dtypes(include=['object']).columns.tolist()
con_int  = df.select_dtypes(include=['int64']).columns.tolist()
con_float  = df.select_dtypes(include=['float64']).columns.tolist()
time = df.select_dtypes(include=['timedelta64']).columns.tolist()

In [336]:
print("Categorical:", cat)
print("Numerical:", con_int,con_float,sep='\n')
print("Time:", time)

Categorical: ['STORE_PRIMARY_CATEGORY']
Numerical:
['STORE_ID', 'TOTAL_ITEMS', 'SUBTOTAL', 'NUM_DISTINCT_ITEMS', 'MIN_ITEM_PRICE', 'MAX_ITEM_PRICE', 'ESTIMATED_ORDER_PLACE_DURATION']
['MARKET_ID', 'ORDER_PROTOCOL', 'TOTAL_ONSHIFT_DASHERS', 'TOTAL_BUSY_DASHERS', 'TOTAL_OUTSTANDING_ORDERS', 'ESTIMATED_STORE_TO_CONSUMER_DRIVING_DURATION', 'DELIVERY_DURATION_SEC']
Time: ['CREATED_AT', 'ACTUAL_DELIVERY_TIME']


In [337]:
# Import Pipeline
from sklearn.pipeline import make_pipeline
# Import Imputer that performs missing data handling by replacing missing data with central tendency
from sklearn.impute import SimpleImputer
# Column Transformer combines pipelines. Here we have 2 pipelines- categorical and continuous 
from sklearn.compose import ColumnTransformer

In [338]:
num_pipe = make_pipeline(SimpleImputer(strategy="constant", fill_value=0))
cat_pipe = make_pipeline(SimpleImputer(strategy="constant", fill_value="Unknown"))
time_pipe = 'passthrough'

In [339]:
# combine both the pipelines
final_pipe = ColumnTransformer(transformers=[
    ('time',time_pipe,time),
    ('cat',cat_pipe,cat),
    ('con_int',num_pipe,con_int),
    ('con_float',num_pipe,con_float)    
]).set_output(transform="pandas")

In [340]:
final_pipe

,transformers,"[('time', ...), ('cat', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'constant'
,fill_value,'Unknown'


In [341]:
df_cleaned = final_pipe.fit_transform(df)
df_cleaned.head(3)

,time__CREATED_AT,time__ACTUAL_DELIVERY_TIME,cat__STORE_PRIMARY_CATEGORY,con_int__STORE_ID,con_int__TOTAL_ITEMS,con_int__SUBTOTAL,con_int__NUM_DISTINCT_ITEMS,con_int__MIN_ITEM_PRICE,con_int__MAX_ITEM_PRICE,con_int__ESTIMATED_ORDER_PLACE_DURATION,con_float__MARKET_ID,con_float__ORDER_PROTOCOL,con_float__TOTAL_ONSHIFT_DASHERS,con_float__TOTAL_BUSY_DASHERS,con_float__TOTAL_OUTSTANDING_ORDERS,con_float__ESTIMATED_STORE_TO_CONSUMER_DRIVING_DURATION,con_float__DELIVERY_DURATION_SEC
0,0 days 22:24:17,0 days 23:27:16,american,1845,4,3441,4,557,1239,446,1.0,1.0,33.0,14.0,21.0,861.0,3779.0
1,0 days 21:49:25,0 days 22:56:29,mexican,5477,1,1900,1,1400,1400,446,2.0,2.0,1.0,2.0,2.0,690.0,4024.0
2,0 days 20:39:28,0 days 21:09:09,Unknown,5477,1,1900,1,1900,1900,446,3.0,1.0,1.0,0.0,0.0,690.0,1781.0


In [342]:
df_cleaned.isna().sum()

time__CREATED_AT                                           0
time__ACTUAL_DELIVERY_TIME                                 0
cat__STORE_PRIMARY_CATEGORY                                0
con_int__STORE_ID                                          0
con_int__TOTAL_ITEMS                                       0
con_int__SUBTOTAL                                          0
con_int__NUM_DISTINCT_ITEMS                                0
con_int__MIN_ITEM_PRICE                                    0
con_int__MAX_ITEM_PRICE                                    0
con_int__ESTIMATED_ORDER_PLACE_DURATION                    0
con_float__MARKET_ID                                       0
con_float__ORDER_PROTOCOL                                  0
con_float__TOTAL_ONSHIFT_DASHERS                           0
con_float__TOTAL_BUSY_DASHERS                              0
con_float__TOTAL_OUTSTANDING_ORDERS                        0
con_float__ESTIMATED_STORE_TO_CONSUMER_DRIVING_DURATION    0
con_float__DELIVERY_DURA

**Now there is no missing data**

In [343]:
new_cols = []
for i in df_cleaned.columns:
    c = i.split('__')[-1]
    new_cols.append(c)
print(new_cols)

['CREATED_AT', 'ACTUAL_DELIVERY_TIME', 'STORE_PRIMARY_CATEGORY', 'STORE_ID', 'TOTAL_ITEMS', 'SUBTOTAL', 'NUM_DISTINCT_ITEMS', 'MIN_ITEM_PRICE', 'MAX_ITEM_PRICE', 'ESTIMATED_ORDER_PLACE_DURATION', 'MARKET_ID', 'ORDER_PROTOCOL', 'TOTAL_ONSHIFT_DASHERS', 'TOTAL_BUSY_DASHERS', 'TOTAL_OUTSTANDING_ORDERS', 'ESTIMATED_STORE_TO_CONSUMER_DRIVING_DURATION', 'DELIVERY_DURATION_SEC']


In [344]:
df_cleaned.columns = new_cols
df_cleaned.columns

Index(['CREATED_AT', 'ACTUAL_DELIVERY_TIME', 'STORE_PRIMARY_CATEGORY',
       'STORE_ID', 'TOTAL_ITEMS', 'SUBTOTAL', 'NUM_DISTINCT_ITEMS',
       'MIN_ITEM_PRICE', 'MAX_ITEM_PRICE', 'ESTIMATED_ORDER_PLACE_DURATION',
       'MARKET_ID', 'ORDER_PROTOCOL', 'TOTAL_ONSHIFT_DASHERS',
       'TOTAL_BUSY_DASHERS', 'TOTAL_OUTSTANDING_ORDERS',
       'ESTIMATED_STORE_TO_CONSUMER_DRIVING_DURATION',
       'DELIVERY_DURATION_SEC'],
      dtype='object')

In [345]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 195402 entries, 0 to 197427
Data columns (total 17 columns):
 #   Column                                        Non-Null Count   Dtype          
---  ------                                        --------------   -----          
 0   CREATED_AT                                    195402 non-null  timedelta64[ns]
 1   ACTUAL_DELIVERY_TIME                          195402 non-null  timedelta64[ns]
 2   STORE_PRIMARY_CATEGORY                        195402 non-null  object         
 3   STORE_ID                                      195402 non-null  int64          
 4   TOTAL_ITEMS                                   195402 non-null  int64          
 5   SUBTOTAL                                      195402 non-null  int64          
 6   NUM_DISTINCT_ITEMS                            195402 non-null  int64          
 7   MIN_ITEM_PRICE                                195402 non-null  int64          
 8   MAX_ITEM_PRICE                                195

### Outliers Detecting & Handling

In [346]:
con = con_int + con_float
con

['STORE_ID',
 'TOTAL_ITEMS',
 'SUBTOTAL',
 'NUM_DISTINCT_ITEMS',
 'MIN_ITEM_PRICE',
 'MAX_ITEM_PRICE',
 'ESTIMATED_ORDER_PLACE_DURATION',
 'MARKET_ID',
 'ORDER_PROTOCOL',
 'TOTAL_ONSHIFT_DASHERS',
 'TOTAL_BUSY_DASHERS',
 'TOTAL_OUTSTANDING_ORDERS',
 'ESTIMATED_STORE_TO_CONSUMER_DRIVING_DURATION',
 'DELIVERY_DURATION_SEC']

In [347]:
# Outliers detecting
detected_outliers= {}
for col in con:

    q1 = np.percentile(df_cleaned[col], 25)
    q3 = np.percentile(df_cleaned[col], 75)

    iqr = q3 - q1 

    minimun = q1 - 1.5 * iqr
    maximum = q3 + 1.5 * iqr

    lower_out = df_cleaned[col] < minimun
    upper_out = df_cleaned[col] > maximum

    outliers = df_cleaned[lower_out | upper_out]

    detected_outliers[col] = {
        "lower": minimun,
        "upper": maximum,
        "ecom": len(outliers),
        "outlier_pct": round(len(outliers)/len(df_cleaned) * 100, 2)
    }

outliers_report = pd.DataFrame(detected_outliers)
outliers_report.T

,lower,upper,ecom,outlier_pct
STORE_ID,-3732.000,10716.000,0.0,0.00
TOTAL_ITEMS,-1.000,7.000,9493.0,4.86
SUBTOTAL,-1594.000,6390.000,8949.0,4.58
NUM_DISTINCT_ITEMS,-2.000,6.000,5872.0,3.01
MIN_ITEM_PRICE,-676.000,1924.000,4557.0,2.33
MAX_ITEM_PRICE,-92.500,2287.500,7660.0,3.92
ESTIMATED_ORDER_PLACE_DURATION,-41.500,738.500,28.0,0.01
MARKET_ID,-1.000,7.000,0.0,0.00
ORDER_PROTOCOL,-3.500,8.500,0.0,0.00
TOTAL_ONSHIFT_DASHERS,-60.500,135.500,1847.0,0.95


In [348]:
# Outliers Handling
def handle_outlier(col):
    global df_outliers
    # 1st quartile - 25%
    q1 = np.percentile(df_cleaned[col],25)
    # 3rd quartile - 75%
    q3 = np.percentile(df_cleaned[col],75)
    # IQR - Inter quartile range
    iqr = q3-q1
    # calculate minimum and maximum to determine outliers
    mini = q1 - (1.5*iqr)
    maxi = q3 + (1.5*iqr)

    lower_index = list(df_cleaned[df_cleaned[col]<mini].index)
    upper_index = list(df_cleaned[df_cleaned[col]>maxi].index)

    all_ = lower_index + upper_index
    df_outliers = df_cleaned.drop(index=all_)
    print("====================================================")
    print(col)
    print(f"Shape before outlier handling is {df_cleaned.shape}")
    print(f"Minimum values present at index {lower_index}")
    print(f"Maximum values present at index {upper_index}")
    print(f"Outliers are handled for {col}")
    print(f"Shape after outlier handling is {df_outliers.shape}")
    print("===============================")
    return df_outliers

In [349]:
for i in con:
    handle_outlier(i)

STORE_ID
Shape before outlier handling is (195402, 17)
Minimum values present at index []
Maximum values present at index []
Outliers are handled for STORE_ID
Shape after outlier handling is (195402, 17)
TOTAL_ITEMS
Shape before outlier handling is (195402, 17)
Minimum values present at index []
Maximum values present at index [37, 59, 122, 136, 165, 180, 193, 242, 246, 258, 283, 352, 382, 391, 458, 460, 486, 506, 532, 536, 558, 560, 576, 628, 634, 638, 659, 678, 696, 719, 795, 798, 800, 814, 967, 975, 982, 990, 1052, 1058, 1064, 1075, 1077, 1084, 1089, 1093, 1096, 1097, 1105, 1209, 1219, 1220, 1222, 1230, 1244, 1284, 1287, 1306, 1342, 1344, 1345, 1346, 1351, 1353, 1355, 1358, 1359, 1360, 1361, 1362, 1368, 1372, 1373, 1375, 1378, 1382, 1384, 1385, 1389, 1486, 1487, 1498, 1509, 1514, 1521, 1524, 1580, 1595, 1607, 1615, 1616, 1661, 1674, 1719, 1728, 1737, 1739, 1746, 1757, 1799, 1811, 1817, 1818, 1820, 1825, 1826, 1827, 1828, 1836, 1840, 1842, 1867, 1868, 1953, 1969, 1970, 2015, 2103, 21

In [350]:
df_outliers.shape

(183610, 17)

In [351]:
df_outliers.to_csv('Historical_Cleaned_Dataset.csv', index=False)

### Statistical Analysis

In [352]:
df_outliers[con].describe().T

,count,mean,std,min,25%,50%,75%,max
STORE_ID,183610.0,3530.465018,2053.336320,1.0,1686.0,3592.0,5299.0,6987.0
TOTAL_ITEMS,183610.0,3.183786,2.661045,1.0,2.0,3.0,4.0,411.0
SUBTOTAL,183610.0,2669.626496,1799.543548,0.0,1400.0,2200.0,3385.0,26800.0
NUM_DISTINCT_ITEMS,183610.0,2.662121,1.616960,1.0,1.0,2.0,3.0,20.0
MIN_ITEM_PRICE,183610.0,686.577006,519.855731,-86.0,299.0,595.0,949.0,14700.0
MAX_ITEM_PRICE,183610.0,1158.026535,554.525776,0.0,800.0,1095.0,1395.0,14700.0
ESTIMATED_ORDER_PLACE_DURATION,183610.0,307.858183,89.678686,0.0,251.0,251.0,446.0,1740.0
MARKET_ID,183610.0,2.991335,1.520131,1.0,2.0,3.0,4.0,6.0
ORDER_PROTOCOL,183610.0,2.885638,1.503135,1.0,1.0,3.0,4.0,7.0
TOTAL_ONSHIFT_DASHERS,183610.0,42.008235,35.502945,-4.0,13.0,34.0,63.0,171.0


In [353]:
df_outliers[cat].describe().T

,count,unique,top,freq
STORE_PRIMARY_CATEGORY,183610,75,american,18120


In [354]:
df_outliers[time].describe().T

,count,mean,std,min,25%,50%,75%,max
CREATED_AT,183610,0 days 08:35:17.299439028,0 days 08:26:01.809215074,0 days 00:00:01,0 days 02:11:07,0 days 03:38:25,0 days 19:25:10.750000,0 days 23:52:55
ACTUAL_DELIVERY_TIME,183610,0 days 09:20:58.441451990,0 days 08:23:24.413476073,0 days 00:15:28,0 days 03:01:38,0 days 04:28:19,0 days 20:08:16,0 days 23:59:59
